download google takeout?



In [ ]:
var importer = require('../Core');var listTakeouts = () => {    return client        .url('https://takeout.google.com/settings/takeout')        .loginGoogle()        .waitUntil(() => client.getUrl()            .then(url => url.indexOf('privacy') > -1 || url.indexOf('takeout') > -1), 20000, '')        .url('https://takeout.google.com/settings/takeout')        .getAllXPath([            '//*[@data-id]',            {                id: './@data-id',                label: './/td[3]//text()'            }        ])};var takeoutProducts = (product) => {    return client        .url('https://takeout.google.com/settings/takeout/custom/' + product.id)        .pause(1000)        .click('//*[contains(@role, "button")][contains(., "Next")]')        .pause(1000)        .execute(() => {            var el = document.evaluate(                '//*[contains(., "Archive size")]/parent::*//*[contains(@role,"option")][contains(.,"2")]',                document,                null,                XPathResult.FIRST_ORDERED_NODE_TYPE, null            ).singleNodeValue;            var event = document.createEvent('MouseEvents');            event.initEvent('mousedown', true, true);            el.dispatchEvent(event, true);        })        .pause(1000)        // TODO: shorten this in to a utility command        // doesn't work because of some weird css BS        // .click('//*[contains(@role, "option")][contains(., "50")]')        .click(            '//*[contains(., "Archive size")]/parent::*//*[contains(@role,"listbox")]/div[3]//*[contains(@role,"option")][contains(.,"50")]')        .click('//*[contains(@role, "button")][contains(., "Create archive")]')        .waitUntil(            () => client.isExisting('//tr[1]/td[last()]//*[contains(@data-download-uri, "takeout/download")]'),            120000,            '')
        .isVisible('iframe[src*="settings"]')
        .then(is => is ? client.execute(() => {
            document.querySelector('iframe[src*="settings"]').remove();
        }) : [])        .click('//tr[1]/td[last()]//*[contains(@data-download-uri, "takeout/download")]')        .pause(2000)
        .isVisible('#passwordNext')
        .then(is => is ? client
            .loginGoogle()
            .click('//tr[1]/td[last()]//*[contains(@data-download-uri, "takeout/download")]')
            .then(is => is ? client.execute(() => {
                document.querySelector('iframe[src*="settings"]').remove();
            }) : [])
             : [])
        .pause(10000)}var downloadGoogleTakeout = (products) => {    var regex = new RegExp(products, 'ig');    return listTakeouts()        .then(ids => {            return ids                .map(k => ({                    id: k.id + '',                    label: k.label + ''                }))                .filter(k => k.id != '_gd' && (products.indexOf('all') > -1                    || k.id.match(regex) != null || k.label.match(regex) != null))        })        .then(selectedProducts => {            console.log(selectedProducts);            return importer.runAllPromises(selectedProducts                .map(p => (resolve) => {                    return takeoutProducts(p)                        .then(r => resolve(r))                        .catch(e => resolve(e))                }))        })};if (typeof client.downloadGoogleTakeout == 'undefined') {    client.addCommand('downloadGoogleTakeout', downloadGoogleTakeout);}module.exports = downloadGoogleTakeout;downloadGoogleTakeout;

order google takeout?



In [ ]:
var importer = require('../Core');var downloadGoogleTakeout;var googleTakeout = (products = 'all?') => {    return importer.import('selenium cell')        .then(runSeleniumCell => runSeleniumCell([            'log in google',            'download google takeout'        ]))        .then(r => {            downloadGoogleTakeout = r[1];            return downloadGoogleTakeout(products);        })        .then(r => {            // TODO: save it in the project path?            return r;        })        .catch(e => console.log(e))};module.exports = googleTakeout;googleTakeout;

test google takeout



In [2]:
var importer = require('../Core');$$.async();importer.import('order google takeout')    .then(googleTakeout => googleTakeout('chrome'))    .then(r => $$.sendResult(r))    .catch(e => $$.sendError(e));

Initializing webdriver on localhost
[ { id: 'chrome', label: 'Chrome' } ]
Google: Require password


[ { Error: An element could not be located on the page using the given search parameters.
    at element("//tr[1]/td[last()]//*[contains(@data-download-uri, "takeout/download")]") - click.js:12:17
    at elementIdClick("0.5142575298731111-1") - click.js:20:22
    type: 'NoSuchElement',
    message: 'An element could not be located on the page using the given search parameters.' } ]

d3 stream my chrome history by category and site sub-divided?



In [ ]:
var fs = require('fs');var importer = require('../Core');var convertUnicode = (x) => {    var r = /\\u([\d\w]{4})/gi;    x = x.replace(r, function (match, grp) {        return String.fromCharCode(parseInt(grp, 16));    });    return unescape(x);}var history = JSON.parse(fs.readFileSync('/Users/briancullinan/Downloads/Takeout 6/Chrome/BrowserHistory.json'));var total = history['Browser History'].length;console.log(new Date(Math.round(history['Browser History'][0].time_usec / 1000)));console.log(new Date(Math.round(history['Browser History'][total - 1].time_usec / 1000)));var groupCounts = [];var addSite = (label, value = 1) => {    var site = groupCounts.filter(g => g.label === label)[0];    if (typeof site === 'undefined') {        groupCounts[groupCounts.length] = {            label: label,            value: value        }    } else {        site.value += value;    }};for (var i = 0; i < total; i++) {    var uri = convertUnicode(history['Browser History'][i].url);    if (uri.match('google.*?search')) {        addSite('google search');    }    else if (uri.match('calendar.*?google')) {        addSite('google calendar');    }    else if (uri.match('stackoverflow')) {        addSite('stackoverflow');    }    else if (uri.match('localhost:9090')) {        addSite('local dev');    }    else if (uri.match('linkedin')) {        addSite('linkedin');    }    else if (uri.match('google')) {        addSite('other google');    }    else if (uri.match('amazon')) {        addSite('amazon');    }    else if (uri.match('github')) {        addSite('github');    }    else if (uri.match('facebook')) {        addSite('facebook');    }    else if (uri.match('wikipedia')) {        addSite('wikipedia');    }    else if (uri.match('youtube')) {        addSite('youtube');    }    else if (uri.match('ftp:')) {        addSite('ftp');    }    else if (uri.match('chrome:|about:|native:|source:|file:|data:|extension:|intent:|content:|market:')) {        addSite('settings');    }    else {        var domain = (/https?:\/\/(.*?)(\/|$)/ig).exec(uri);        if (domain === null) {            console.log(uri);        }        addSite(domain[1]);    }}var oldGroups = groupCounts;groupCounts = [];for (var i = 0; i < oldGroups.length; i++) {    if (oldGroups[i].value < 300) {        addSite('other', oldGroups[i].value);    } else {        addSite(oldGroups[i].label, oldGroups[i].value);    }}$$.async();importer.import('d3 pie chart')    .then(d3PieChart => d3PieChart(groupCounts))    .then(r => $$.svg(r))    .catch(e => $$.sendError(e));// TODO: show month by month, week by week?// TODO: use this chart instead? https://bl.ocks.org/mbostock/8d2112a115ad95f4a6848001389182fb